<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


<h1 align="center">Open Data QnA - Chat with your SQL Database</h1> 

---

This notebook first walks through the Vector Store Setup needed for running the Open Data QnA application. 

Currently supported Source DBs are: 
- PostgreSQL on Google Cloud SQL 
- BigQuery

Furthermore, the following vector stores are supported 
- pgvector on PostgreSQL 
- BigQuery vector


The setup part covers the following steps: 
> 1. Configuration: Intial GCP project, IAM permissions, Environment  and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)

> 3. Setting up firestore for persisting the session history for multiturn


Afterwards, you will be able to run the Open Data QnA Pipeline to generate SQL queries and answer questions over your data source. 

### 📒 Using this interactive notebook

If you have not used this IDE with jupyter notebooks it will ask for installing Python + Jupyter extensions. Please go ahead install them

Click the **run** icons ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

### **Change your Kernel to the created .venv with poetry from README.md**

Path would look like e.g. /home/admin_/opendata/.venv/bin/python or ~cache/user/opendataqna/.venv/bin/python

Below is the Kernel how it should look like before you proceed.


![Kernel](../utilities/imgs/Kernel%20Changed.png)

### Set Python Module Path to Root

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

### 🔗 **Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [ ]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = input("Enter the project id (same as your Setup Project) to copy source data in bigquery for this solution")

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

#If errors out for authentication restart the kernel and start from the previous cell

### ⚙️ **Enable Required API Services in the GCP Project**

In [ ]:
#Enable all the required APIs for the Open Data QnA solution

!gcloud services enable \
  cloudapis.googleapis.com \
  compute.googleapis.com \
  iam.googleapis.com \
  run.googleapis.com \
  sqladmin.googleapis.com \
  aiplatform.googleapis.com \
  bigquery.googleapis.com \
  firestore.googleapis.com --project {PROJECT_ID}

## **Configure your inputs for the environments**

This section assumes that a datasource is already set up in your GCP project. If a datasource has not been set up, use the notebooks below to copy a public data set from BigQuery to Cloud SQL or BigQuery on your GCP project


Enabled Data Sources:
* PostgreSQL on Google Cloud SQL (Copy Sample Data: [0_CopyDataToCloudSqlPG.ipynb](0_CopyDataToCloudSqlPG.ipynb))
* BigQuery (Copy Sample Data: [0_CopyDataToBigQuery.ipynb](0_CopyDataToBigQuery.ipynb))

Enabled Vector Stores:
* pgvector on PostgreSQL 
* BigQuery vector


### 🤔 **Choose Data Source and Vector Store**

**Sources to connect**
- This solution lets you setup multiple data source at the same time.
- You can group multiple tables from different datasets or schema into a grouping and provide the details
- If your dataset/schema has many tables and you want to run the solution against few you should specifically choose a group for that tables only

**Format for data_source_list.csv**

**source | user_grouping | schema | table**

**source** - Supported Data Sources. #Options: bigquery , cloudsql-pg

**user_grouping** - Logical grouping or use case name for tables from same or different schema/dataset. When left black it default to the schema value in the next column

**schema** - schema name for postgres or dataset name in bigquery 

**table** - name of the tables to run the solutions against. Leave this column blank after filling schema/dataset if you want to run solution for whole dataset/schema


Fill out the parameters and configuration settings below. 
These are the parameters for setting configurations for the vector store tables to be created. 

Additionally, you can specify whether you have and want to use known-good-queries for the pipeline run and whether you want to enable logging.

**Known good queries:** if you have known working user question <-> SQL query pairs, you can put them into the file `scripts/known_good_sql.csv`. This will be used as a caching layer and for in-context learning: If an exact match of the user question is found in the vector store, the pipeline will skip SQL Generation and output the cached SQL query. If the similarity score is between 90-100%, the known good queries will be used as few-shot examples by the SQL Generator Agent. 

**Logging:** you can enable logging. If enabled, a dataset is created in Big Query in your project, which will store the logging table and save information from the pipeline run in the logging table. This is especially helpful for debugging.

**use_column_samples:** you can enable use column samples flag to let the pipeline select sample values of the columns from the source database. In some specific usecase where we need to understand the format or case sensitivity of the values this flag help LLM to have better understanding. Though this is one time setup, please be aware that turning this on mean getting samples from each column and it can be an expensive operation when there are lot many columns.

In [ ]:
#[CONFIG]
embedding_model = 'vertex' # Options: 'vertex' or 'vertex-lang'
description_model = 'gemini-1.5-pro' # 'gemini-1.0-pro', 'gemini-1.5-pro', 'text-bison-32k', 'gemini-1.5-flash'
vector_store = 'bigquery-vector' # Options: 'bigquery-vector', 'cloudsql-pgvector'
logging = True # True or False 
kgq_examples = True # True or False
use_column_samples = True #True or False

#[GCP]
project_id = PROJECT_ID

#[PGCLOUDSQL]
# Default values for pgvector setup, change only if needed
pg_region = 'us-central1'
pg_instance = 'pg15-opendataqna'
pg_database = 'opendataqna-db'
pg_user = 'pguser'
pg_password = 'pg123'

#[BIGQUERY]
# Name for the BQ dataset created for bigquery-vector and/or logging. Change names only if needed.
bq_dataset_region = 'us-central1'
bq_opendataqna_dataset_name = 'opendataqna'
bq_log_table_name = 'audit_log_table'

#Details for firestore to store the chat session history
firestore_region='us-central1'
## firestore_database is named as 'opendataqna-session-logs' (This is designed to not be customizable as the setup includes creation of composite indexes from backend)

Quick input verifications below:

In [ ]:

# Input verification - Vector Store
assert vector_store in {'bigquery-vector', 'cloudsql-pgvector'}, "⚠️ Invalid VECTOR_STORE. Must be 'bigquery-vector' or 'cloudsql-pgvector'"

# Input verification - Firestore Region
assert firestore_region, "⚠️ Provide firestore region name"

if logging: 
    assert bq_log_table_name, "⚠️ Please provide a name for your log table if you want to use logging"

if vector_store == 'bigquery':
    assert bq_dataset_region, "⚠️ Please provide the Data Set Region"
    assert bq_opendataqna_dataset_name, "⚠️ Please provide the name of the logging/vector store dataset on Bigquery"

elif vector_store == 'cloudsql-pg':
    assert pg_region, "⚠️ Please provide Region of the Cloud SQL Instance"
    assert pg_instance, "⚠️ Please provide the name of the Cloud SQL Instance"
    assert pg_database, "⚠️ Please provide the name of the PostgreSQL Database on the Cloud SQL Instance"
    assert pg_user, "⚠️ Please provide a username for the Cloud SQL Instance"
    assert pg_password, "⚠️ Please provide the Password for the PG_USER"


### 💾 **Save Configuration to File** 
Save the configurations set in this notebook to  `config.ini`. The parameters from this file are used in notebooks and in various modeules in the repo

In [ ]:
from scripts import save_config

save_config(embedding_model, description_model, vector_store, logging, kgq_examples, use_column_samples, PROJECT_ID,
            pg_region, pg_instance, pg_database, pg_user, pg_password, 
            bq_dataset_region, bq_opendataqna_dataset_name, bq_log_table_name,firestore_region)

# **1. Vector Store Setup** (Run once)
---

This section walks through the Vector Store Setup needed for running the Open Data QnA application. 

It covers the following steps: 
> 1. Configuration: Environment and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)




## ⚙️ **1.1 Database Setup for Vector Store**

If BigQuery is your vector store, the dataset is created.

If 'cloudsql-pgvector' is chosen as vector store, PostgreSQL Instance on CloudSQL (Note that this version of code supports only creating vector store on same instance as source)

The cell will also create a dataset to store the log table on Big Query, **if** logging is enabled if its not bigquery vector.

In [ ]:
from env_setup import create_vector_store
# Setup vector store for embeddings
create_vector_store()

##  **1.2 Create Embeddings in Vector Store for RAG** 

### 🖋️ **Create Table and Column Embeddings**

In this step, table and column metadata is retreived from the data source and embeddings are generated for both

In [ ]:
from env_setup import get_embeddings

# Set generate_missing_descriptions_flag = True for LLMs to getenrate table/column descriptions when missing
# False otherwise.
# generate_missing_descriptions_flag defaults to False
# Set sleep_time based on your quota to avoid quota limit errors, 
# sleep_time can be >= 0 and defaults to 1, skips sleep when sleep_time = 0
# sleep_time is immaterial when generate_missing_descriptions_flag = False

# Generate embeddings for tables and columns
table_schema_embeddings, col_schema_embeddings = get_embeddings(generate_missing_descriptions_flag = False, sleep_time = 0)


### 💾 **Save the Table and Column Embeddings in the Vector Store**
The table and column embeddings created in the above step are save to the Vector Store chosen

In [ ]:
from env_setup import store_embeddings

# Store table/column embeddings (asynchronous)
await(store_embeddings(table_schema_embeddings, col_schema_embeddings))


# **2. Firestore Database Setup**
---

This section walks through setting up the firestore DB to store the session history of the conversation for multiturn

It covers the following steps: 
> 1. Creation Firestore Database



In [ ]:
from env_setup import create_firestore_db

create_firestore_db(firestore_region)

## 🗄️ **3. Load Known Good SQL into Vector Store**
Known Good Queries are used to create query cache for Few shot examples. Creating a query cache is highly recommended for best outcomes! 

The following cell will load the Natural Language Question and Known Good SQL pairs into our Vector Store. There pairs are loaded from `known_good_sql.csv` file inside scripts folder. If you have your own Question-SQL examples, curate them in .csv file before running the cell below. 

If no Known Good Queries are available at this time to create query cache, you can use [3_LoadKnownGoodSQL.ipynb](3_LoadKnownGoodSQL.ipynb) to load them later. 



#### Format of the Known Good SQL File (known_good_sql.csv)

prompt | sql | user_grouping [3 columns]

prompt ==> User Question 

sql ==> SQL for the user question (Note that the sql should enclosed in quotes and only in single line. Please remove the line  break)

user_grouping ==>This name should exactly  match the grouping name you mentioned while creating vector store

In [ ]:
from env_setup import create_kgq_sql_table, store_kgq_sql_embeddings

# Create table for known good queries (if enabled)
await(create_kgq_sql_table()) 

# Store known good query embeddings (if enabled)
await(store_kgq_sql_embeddings())  


### 🥁 If all the above steps are executed suucessfully, the following should be set up:

* GCP project and all the required IAM permissions

* Environment to run the solution

* Data source and Vector store for the solution

__________________________________________________________________________________________________________________